In [1]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np

In [2]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

In [3]:
req=requests.get('http://steamspy.com/api.php?request=top100forever')
req.content

b'{"570":{"appid":570,"name":"Dota 2","developer":"Valve","publisher":"Valve","score_rank":"","positive":1434815,"negative":284839,"userscore":0,"owners":"100,000,000 .. 200,000,000","average_forever":38155,"average_2weeks":1565,"median_forever":1079,"median_2weeks":927,"price":"0","initialprice":"0","discount":"0","ccu":551497},\n"730":{"appid":730,"name":"Counter-Strike: Global Offensive","developer":"Valve, Hidden Path Entertainment","publisher":"Valve","score_rank":"","positive":5609349,"negative":748640,"userscore":0,"owners":"50,000,000 .. 100,000,000","average_forever":30492,"average_2weeks":784,"median_forever":7775,"median_2weeks":260,"price":"0","initialprice":"0","discount":"0","ccu":755785},\n"578080":{"appid":578080,"name":"PUBG: BATTLEGROUNDS","developer":"KRAFTON, Inc.","publisher":"KRAFTON, Inc.","score_rank":"","positive":1123879,"negative":882085,"userscore":0,"owners":"50,000,000 .. 100,000,000","average_forever":22834,"average_2weeks":830,"median_forever":8275,"medi

In [4]:
dic_steam = req.json()   ##############  To make requests.models.Response become dict   ###############
dic_steam

{'10': {'appid': 10,
  'average_2weeks': 323,
  'average_forever': 11137,
  'ccu': 14091,
  'developer': 'Valve',
  'discount': '0',
  'initialprice': '999',
  'median_2weeks': 355,
  'median_forever': 190,
  'name': 'Counter-Strike',
  'negative': 5021,
  'owners': '10,000,000 .. 20,000,000',
  'positive': 195401,
  'price': '999',
  'publisher': 'Valve',
  'score_rank': '',
  'userscore': 0},
 '10180': {'appid': 10180,
  'average_2weeks': 36,
  'average_forever': 1247,
  'ccu': 626,
  'developer': 'Infinity Ward',
  'discount': '25',
  'initialprice': '1999',
  'median_2weeks': 55,
  'median_forever': 467,
  'name': 'Call of Duty: Modern Warfare 2',
  'negative': 3250,
  'owners': '5,000,000 .. 10,000,000',
  'positive': 43111,
  'price': '1499',
  'publisher': 'Activision',
  'score_rank': '',
  'userscore': 0},
 '1046930': {'appid': 1046930,
  'average_2weeks': 227,
  'average_forever': 1768,
  'ccu': 2835,
  'developer': 'Valve',
  'discount': '0',
  'initialprice': '0',
  'median

In [5]:
total_count= 0
with open('app_steam.csv','w') as part_two:
    for game_id,game_details in dic_steam.items():
        total_count =total_count+1
        part_two.write(json.dumps({game_id:game_details}))
        part_two.write('\n')

In [6]:
dic_game_genre = {'appid':{},'name':{},'positive':{},'negative':{}}
current_count = 0
for game_id,game_details in dic_steam.items():
    if game_details!={} and game_details is not None:
        dic_game_genre['appid'].update({game_id:game_details.get("appid",{})})
        dic_game_genre['name'].update({game_id:game_details.get('name',{})})
        dic_game_genre['positive'].update({game_id:game_details.get('positive',{})})
        dic_game_genre['negative'].update({game_id:game_details.get('negative',{})})
    show_work_status(1,total_count,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [7]:
pos = list(dic_game_genre.get('positive').values())
neg = list(dic_game_genre.get('negative').values())
appid = list(dic_game_genre.get('appid').values())
name = list(dic_game_genre.get('name').values())

In [8]:
content_data = pd.DataFrame(
    {'Positive': pos,
     'Negative': neg,
     'Game_id': appid,
     'Game_name': name,
    })

content_data['Positive_Rate'] =  content_data["Positive"]/(content_data["Positive"]+content_data["Negative"])
content_data.sort_values("Positive_Rate", axis = 0, ascending = False, 
                 inplace = True, na_position ='first') 
content_data = content_data.reset_index(drop=True)
content_data

,Positive,Negative,Game_id,Game_name,Positive_Rate
0,288896,3509,620,Portal 2,0.988000
1,104166,1657,400,Portal,0.984342
2,447749,8690,413150,Stardew Valley,0.980961
3,489262,9868,431960,Wallpaper Engine,0.980230
4,924124,19812,105600,Terraria,0.979011
...,...,...,...,...,...
95,1123879,882085,578080,PUBG: BATTLEGROUNDS,0.560269
96,115588,92958,433850,Z1 Battle Royale,0.554257
97,1313,1176,439700,Z1 Battle Royale: Test Server,0.527521
98,30506,30083,1089350,NBA 2K20,0.503491


In [9]:
content_data['Game_id'] = content_data['Game_id'].apply(str)  ########## Convert to String, Using them later  ############
content_data['links'] = ('https://store.steampowered.com/app/'+ content_data['Game_id']).astype(str)
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Positive_Rate,links
0,288896,3509,620,Portal 2,0.988000,https://store.steampowered.com/app/620
1,104166,1657,400,Portal,0.984342,https://store.steampowered.com/app/400
2,447749,8690,413150,Stardew Valley,0.980961,https://store.steampowered.com/app/413150
3,489262,9868,431960,Wallpaper Engine,0.980230,https://store.steampowered.com/app/431960
4,924124,19812,105600,Terraria,0.979011,https://store.steampowered.com/app/105600


In [10]:
import urllib
import bs4
from bs4 import BeautifulSoup
#url_contents = urllib.request.urlopen(url).read()
Game_description = pd.Series([]) 
current_count = 0
for i in range(100):
    #print(content_data['links'][i])
    url = content_data['links'][i]
    url_contents = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(url_contents, 'html.parser')
    div = soup.find("div", {"id": "game_area_description"})
    ########## Fina the area of game descriptiona ##########
    Game_description[i]=div
    show_work_status(1,100,current_count)
    current_count+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [43]:
soup = bs4.BeautifulSoup(url_contents, 'html.parser')
div = soup.find("div", {"id": "game_area_description"})

content = str(div)
content_data['Game_description'] = Game_description
content_data['Game_description'] = content_data['Game_description'].apply(str)
content_data

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description
0,288896,3509,620,Portal 2,0.988000,https://store.steampowered.com/app/620,"<div class=""game_area_description"" id=""game_ar..."
1,104166,1657,400,Portal,0.984342,https://store.steampowered.com/app/400,"<div class=""game_area_description"" id=""game_ar..."
2,447749,8690,413150,Stardew Valley,0.980961,https://store.steampowered.com/app/413150,"<div class=""game_area_description"" id=""game_ar..."
3,489262,9868,431960,Wallpaper Engine,0.980230,https://store.steampowered.com/app/431960,"<div class=""game_area_description"" id=""game_ar..."
4,924124,19812,105600,Terraria,0.979011,https://store.steampowered.com/app/105600,"<div class=""game_area_description"" id=""game_ar..."
...,...,...,...,...,...,...,...
95,1123879,882085,578080,PUBG: BATTLEGROUNDS,0.560269,https://store.steampowered.com/app/578080,"<div class=""game_area_description"" id=""game_ar..."
96,115588,92958,433850,Z1 Battle Royale,0.554257,https://store.steampowered.com/app/433850,"<div class=""game_area_description"" id=""game_ar..."
97,1313,1176,439700,Z1 Battle Royale: Test Server,0.527521,https://store.steampowered.com/app/439700,"<div class=""game_area_description"" id=""game_ar..."
98,30506,30083,1089350,NBA 2K20,0.503491,https://store.steampowered.com/app/1089350,"<div class=""game_area_description"" id=""game_ar..."


In [44]:
print(content_data['Game_description'][1])
#content_data['Game_description'][1] = re.sub('<.*?>', ' ', content_data['Game_description'][1])
#print(content_data['Game_description'][1])

<div class="game_area_description" id="game_area_description">
<h2>About This Game</h2>
<p>Portal™ is a new single player game from Valve. Set in the mysterious Aperture Science Laboratories, Portal has been called one of the most innovative new games on the horizon and will offer gamers hours of unique gameplay.</p>
<p>The game is designed to change the way players approach, manipulate, and surmise the possibilities in a given environment; similar to how Half-Life® 2's Gravity Gun innovated new ways to leverage an object in any given situation.</p>
<p>Players must solve physical puzzles and challenges by opening portals to maneuvering objects, and themselves, through space.</p>
</div>


In [53]:
import string
current_count = 0
for i in range(0,100):
    content_data['Game_description'][i] = re.sub('<.*?>', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = re.sub('\\n.*\\n', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = content_data['Game_description'][i].replace("\t","")
    content_data['Game_description'][i] = content_data['Game_description'][i].translate(content_data['Game_description'][i].maketrans(' ',' ',string.punctuation))

    show_work_status(1,100,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>---------------------------]47.00%

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: Se

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [54]:
content_data.head(5)

,Positive,Negative,Game_id,Game_name,Positive_Rate,links,Game_description
0,288896,3509,620,Portal 2,0.988000,https://store.steampowered.com/app/620,Portal 2 draws from the awardwinning formula...
1,104166,1657,400,Portal,0.984342,https://store.steampowered.com/app/400,Portal™ is a new single player game from Va...
2,447749,8690,413150,Stardew Valley,0.980961,https://store.steampowered.com/app/413150,Stardew Valley is an openended countrylife R...
3,489262,9868,431960,Wallpaper Engine,0.980230,https://store.steampowered.com/app/431960,Wallpaper Engine enables you to use live wal...
4,924124,19812,105600,Terraria,0.979011,https://store.steampowered.com/app/105600,Dig Fight Explore Build The very world is a...


In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(list(content_data['Game_description']))
#Need to convert every Series in DataFrame to List format
tfidf_vectorizer.get_feature_names()
#Array mapping from feature integer indices to feature name
#print(tfidf)  ####Tf-idf-weighted document-term matrix.     (0, 3473)	0.4059439800797409   (0, 1417)	0.08671197437269683

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10',
 '100',
 '1000th',
 '10th',
 '11',
 '110',
 '111',
 '12',
 '12312021',
 '128',
 '13',
 '1300',
 '14',
 '140',
 '15',
 '150',
 '1500s',
 '1610',
 '169',
 '17',
 '19',
 '1942',
 '1998',
 '1999',
 '1realistic',
 '1v1',
 '1v3',
 '20',
 '200',
 '2000',
 '20000',
 '2001',
 '200203',
 '2003',
 '2006',
 '2007',
 '2007s',
 '2008',
 '2009',
 '200910',
 '2011',
 '201314',
 '2015',
 '201516',
 '2019',
 '2020',
 '2042',
 '2077',
 '20mm',
 '20th',
 '219',
 '22',
 '25',
 '250',
 '26',
 '290',
 '2d',
 '2k',
 '2k20',
 '2ktv',
 '2player',
 '2ton',
 '2v2',
 '2various',
 '2vs2',
 '30',
 '300',
 '300000',
 '320x240',
 '35',
 '3d',
 '3rd',
 '3unique',
 '40',
 '400',
 '4000',
 '415',
 '43',
 '460',
 '48',
 '499',
 '4k',
 '4player',
 '4v4',
 '4v4s',
 '4vs1',
 '4x4',
 '50',
 '500',
 '550',
 '57',
 '60',
 '60000',
 '60man',
 '64bit',
 '65',
 '6th',
 '70',
 '75',
 '7vs7',
 '800',
 '85',
 '87',
 '8person',
 '8th',
 '8v8',
 '90s',
 'abandoned',
 'abilities',
 'ability',
 'ablaze',
 'able',
 'abound',
 'abov

In [62]:
list_app_name = list(content_data['Game_name'])
content_data['Game_description'] 

0       Portal 2 draws from the awardwinning formula...
1        Portal™ is a new single player game from Va...
2       Stardew Valley is an openended countrylife R...
3       Wallpaper Engine enables you to use live wal...
4       Dig Fight Explore Build  The very world is a...
                            ...                        
95       LAND LOOT SURVIVE  Play PUBG BATTLEGROUNDS ...
96      Z1 Battle Royale is a Free to Play fastpaced...
97        TEST SERVER   Z1 Battle Royale is a Free t...
98      NOTE All multiplayer servers for NBA 2K20 wi...
99       WELCOME TO 2042 Battlefield™ 2042 is a firs...
Name: Game_description, Length: 100, dtype: object

In [98]:
dic_recommended_1 = {}
for index in range(content_data['Game_description'].shape[0]):
    #Iterate items in DataFrame
    similarities_1 = linear_kernel(tfidf[index],tfidf).flatten()
    #print(tfidf)  ####Tf-idf-weighted document-term matrix.     (0, 3473)	0.4059439800797409   (0, 1417)	0.08671197437269683
    #print(linear_kernel(tfidf[1],tfidf[3]))
    ####For each game，tfidf value list is a very long list，tfidf[index:index+1] calculates all of the words'tfidf for one game,
    
    #similarities_2 = cosine_similarities(tfidf[index:index+1],tfidf).flatten()
    related_docs_indices_1 = (-similarities_1).argsort()[1:11]
    #related_docs_indices_2 = similarities_2.argsort()[:-10:-1]

   
    #argsort returns an array of indices of the same shape as 
    dic_recommended_1.update({list_app_name[index]:[list_app_name[i] for i in related_docs_indices_1]})
df_content_based_results_1 = pd.DataFrame(dic_recommended_1)
#df_content_based_results_1.index.name = 'steam_appname'
df_content_based_results_1.reset_index(inplace=True)

In [99]:
df_content_based_results_1 = df_content_based_results_1.T
print(df_content_based_results_1.shape)
df_content_based_results_1

(101, 10)


,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
Portal 2,Portal,Half-Life,Call of Duty: Modern Warfare 2,Left 4 Dead 2,Grand Theft Auto V,Counter-Strike: Condition Zero,Borderlands 2,Black Squad,A Story About My Uncle,Dying Light
Portal,Portal 2,Counter-Strike: Condition Zero,Counter-Strike: Global Offensive,Call of Duty: Modern Warfare 2,Left 4 Dead 2,Borderlands 2,Sid Meier's Civilization V,The Tiny Bang Story,Stardew Valley,Human: Fall Flat
Stardew Valley,7 Days to Die,Left 4 Dead 2,New World,ARK: Survival Evolved,Valheim,Grim Dawn,Risk of Rain 2,Mount & Blade II: Bannerlord,The Elder Scrolls V: Skyrim,Apex Legends
Wallpaper Engine,Dying Light,ARK: Survival Evolved,Unturned,Human: Fall Flat,Euro Truck Simulator 2,The Elder Scrolls V: Skyrim,Brawlhalla,World of Warships,Guacamelee! Super Turbo Championship Edition,Cities: Skylines
...,...,...,...,...,...,...,...,...,...,...
PUBG: BATTLEGROUNDS,Apex Legends,Z1 Battle Royale,Z1 Battle Royale: Test Server,7 Days to Die,Unturned,Battlefield 2042,Black Squad,World of Tanks Blitz,World of Warships,New World
Z1 Battle Royale,Z1 Battle Royale: Test Server,Black Squad,Dota Underlords,Brawlhalla,PUBG: BATTLEGROUNDS,Apex Legends,Dirty Bomb,Robocraft,Tom Clancy's Rainbow Six Siege,Call of Duty: Modern Warfare 2
Z1 Battle Royale: Test Server,Z1 Battle Royale,Black Squad,Apex Legends,Dota Underlords,PUBG: BATTLEGROUNDS,Brawlhalla,Tom Clancy's Rainbow Six Siege,Left 4 Dead 2,NBA 2K20,Borderlands 2
NBA 2K20,Sid Meier's Civilization V,Call of Duty: Modern Warfare 2,Apex Legends,Halo Infinite,Black Squad,Team Fortress 2,Brawlhalla,Portal 2,Counter-Strike Nexon: Studio,Left 4 Dead 2


In [52]:
import string
name = "Portal"
id = content_data.loc[content_data['Game_name'] == name]['Game_id'].values[0]
url = 'https://store.steampowered.com/app/' + id
url_content = urllib.request.urlopen(url).read()
soup = bs4.BeautifulSoup(url_content, 'html.parser')
descirption = str(soup.find("div", {"id": "game_area_description"}))
descirption = re.sub('<.*?>', ' ', descirption)
descirption = re.sub('\\n.*\\n', ' ', descirption)
descirption = descirption.replace("\t","")
descirption = descirption.translate(descirption.maketrans(' ',' ',string.punctuation))
print(descirption)

   Portal™ is a new single player game from Valve Set in the mysterious Aperture Science Laboratories Portal has been called one of the most innovative new games on the horizon and will offer gamers hours of unique gameplay   Players must solve physical puzzles and challenges by opening portals to maneuvering objects and themselves through space 
 


In [106]:
print(content_data['Game_name'].tolist())

['Portal 2', 'Portal', 'Stardew Valley', 'Wallpaper Engine', 'Terraria', 'Counter-Strike', 'Left 4 Dead 2', 'Half-Life 2', 'Euro Truck Simulator 2', 'Phasmophobia', "Garry's Mod", 'Half-Life', 'Risk of Rain 2', 'Tomb Raider', 'Counter-Strike: Source', "Sid Meier's Civilization V", 'The Witcher 3: Wild Hunt', 'Valheim', 'The Forest', 'Dying Light', 'Sekiro: Shadows Die Twice - GOTY Edition', 'The Elder Scrolls V: Skyrim', 'Human: Fall Flat', 'DARK SOULS III', 'Borderlands 2', 'Team Fortress 2', 'Grim Dawn', 'Cities: Skylines', 'Raft', 'Call of Duty: Modern Warfare 2', 'Guacamelee! Super Turbo Championship Edition', 'Kathy Rain', 'Among Us', 'A Story About My Uncle', 'Unturned', 'Half-Life: Blue Shift', 'Counter-Strike: Condition Zero', 'Path of Exile', 'Warframe', 'Arma 3', 'Half-Life 2: Deathmatch', 'No More Room in Hell', 'ELDEN RING', 'PAYDAY 2', 'Rocket League', 'Fallout Shelter', 'Counter-Strike: Global Offensive', '7 Days to Die', 'Half-Life 2: Lost Coast', 'Mount & Blade II: Bann

Example Output

In [127]:
user_profile = [["DARK SOULS III", 286], ['Sekiro: Shadows Die Twice - GOTY Edition', 45], ['The Witcher 3: Wild Hunt', 8], ['ELDEN RING', 88], ['Warface', 1.4]]


In [128]:
names = []
for i in range(len(user_profile)):
  names.append(user_profile[i][0])
  if user_profile[i][1] <= 2:
    user_profile[i][1] = 0.1
  elif user_profile[i][1] > 2 and user_profile[i][1] <= 10:
    user_profile[i][1] = 0.5
  elif user_profile[i][1] > 10 and user_profile[i][1] <= 50:
    user_profile[i][1] = 0.8
  elif user_profile[i][1] > 50 and user_profile[i][1] <= 100: 
    user_profile[i][1] = 1
  else:
    user_profile[i][1] = 1.2
user_profile

[['DARK SOULS III', 1.2],
 ['Sekiro: Shadows Die Twice - GOTY Edition', 0.8],
 ['The Witcher 3: Wild Hunt', 0.5],
 ['ELDEN RING', 1],
 ['Warface', 0.1]]

In [138]:
recommendations = {}
for game in user_profile:
  name = game[0]
  id = content_data.index[content_data['Game_name'] == name][0]
  similarities = linear_kernel(tfidf[id],tfidf).flatten()
  related_docs_indices = (-similarities).argsort()[1:11]
  lam = 0
  for i in related_docs_indices:
    new_game = list_app_name[i]
    if new_game not in names:
      if new_game not in recommendations:
        recommendations.update({new_game: game[1]-lam})
      else:
        value = recommendations.get(new_game)
        recommendations.update({new_game: value+game[1]-lam})
    lam += 0.01

recommendations = dict(sorted(recommendations.items(), key=lambda item: item[1], reverse=True))
print(recommendations)

{'The Elder Scrolls V: Skyrim': 3.37, 'Lost Ark': 2.7399999999999998, 'Valheim': 2.55, '7 Days to Die': 2.54, 'Dying Light': 2.36, 'Call of Duty: Modern Warfare 2': 1.93, 'Path of Exile': 1.68, 'Destiny 2': 1.26, 'Half-Life 2: Lost Coast': 1.15, 'A Story About My Uncle': 1.14, 'Guacamelee! Super Turbo Championship Edition': 1.1199999999999999, 'Brawlhalla': 1.02, 'Street Warriors Online': 0.99, 'Mount & Blade II: Bannerlord': 0.96, 'KurtzPel': 0.9400000000000001, 'Half-Life': 0.8, 'Ring of Elysium': 0.74, 'Tomb Raider': 0.73, 'No More Room in Hell': 0.7200000000000001, 'Monster Hunter: World': 0.5, 'New World': 0.47, 'Fallout 4': 0.45, '鬼谷八荒 Tale of Immortal': 0.42, 'Black Squad': 0.09000000000000001, 'PlanetSide 2': 0.060000000000000005, 'War Thunder': 0.05, 'Counter-Strike Nexon: Studio': 0.04, 'Team Fortress 2': 0.03}


In [139]:
output = []
count = 0
for key in recommendations:
  output.append(key)
  count += 1
  if count >= 10:
    break
output

['The Elder Scrolls V: Skyrim',
 'Lost Ark',
 'Valheim',
 '7 Days to Die',
 'Dying Light',
 'Call of Duty: Modern Warfare 2',
 'Path of Exile',
 'Destiny 2',
 'Half-Life 2: Lost Coast',
 'A Story About My Uncle']